### General idea
1. Clean dataset => dfClean
2. Vectorize words => to probability density
3. Perform logistic regression on vectorized words 
    of scales of reviews 0 (0,.1),1 (.2,.3) ,2 (.4,.5),3 (.6,.7) ,4 (.8,.9, 1) reviews

In [75]:
# Read data set and stop words
import pandas as pd
import re 
import nltk
#nltk.download('stopwords')
df1111 = pd.read_csv (r'.\AppReview.csv')
#df=df1.sample(n=1000)
len(df111.index)

df1111


,appID,reviewerName,reviewText,reviewerRating,reviewDate,textAnalytics
0,3,Eric Hansen,Love it! WELL worth the money for the full ver...,1.0,2017/07/07 00:00:00,NaN
1,3,Jacob N.,There's an awful bug that doesn't allow you to...,0.4,2017/08/29 00:00:00,NaN
2,3,Higgins Family,Would be 5 stars except for the bugs.... For e...,0.8,2017/10/02 00:00:00,NaN
3,3,Rajko Dikmann,Worked perfect until a few weeks ago. Then bro...,0.4,2017/09/28 00:00:00,NaN
4,3,Sergei Garcia,Hands down the best browser on the play store!...,1.0,2017/07/09 00:00:00,NaN
...,...,...,...,...,...,...
111138,343,heera d,good less power consumer Full Review,0.8,1-September-2013,NaN
111139,343,Maulik Upadhyay,Xperia pro Nice...but some change would be nic...,0.8,21-August-2013,NaN
111140,343,Lezlie Coleman,O Willl no Full Review,0.6,24-September-2013,NaN
111141,343,Taher Bhai,Great Wonderful app in samsung galaxy y Full R...,1.0,14-August-2013,NaN


In [77]:
df111=df1111[df111["reviewerName"]!="A Google User"]
len(df111.index)

C:\Users\danxg\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [2]:
df111.groupby("reviewerRating").nunique()

,appID,reviewerName,reviewText,reviewerRating,reviewDate,textAnalytics
reviewerRating,,,,,,
0.0,18,1,71,1,60,1
0.2,338,7763,9911,1,3857,24
0.4,294,4012,5134,1,2852,17
0.6,329,6965,8883,1,3759,17
0.8,386,14010,17645,1,4963,25
1.0,473,49121,59953,1,5957,41


In [36]:
frames = []
for i in range(0,12,2):
    frames.append(df111[df111["reviewerRating"]==i/10].sample(n=2000, replace=True))

df=pd.concat(frames,ignore_index =True)
df.groupby("reviewerRating").nunique()
len(df.index)
df

,appID,reviewerName,reviewText,reviewerRating,reviewDate,textAnalytics
0,2297,A Google User,What do i put in the imput box that requires l...,0.0,27-January-2011,-9.223372e+18
1,2123,A Google User,Not recommend Full Review,0.0,10-August-2010,-9.223372e+18
2,2078,A Google User,Great. Game played it for hours. Have now beat...,0.0,20-January-2011,-9.223372e+18
3,2078,A Google User,Love the game. Level 65 is killing me. Any sug...,0.0,3-January-2011,-9.223372e+18
4,194,A Google User,Constantly force close. Please fix. I used tob...,0.0,14-February-2011,-9.223372e+18
...,...,...,...,...,...,...
11995,109,A Google User,Love that you can view in top taskbar without ...,1.0,17-July-2011,-9.223372e+18
11996,1758,A Google User,Awesome Just what I needed! Full Review,1.0,2012/07/10 00:00:00,-9.223372e+18
11997,1888,B. Miller,Awesome app Works great Full Review,1.0,2015/11/04 00:00:00,-9.223372e+18
11998,1061,Tumma Goutham,Really awesome Full Review,1.0,2017/08/26 00:00:00,-9.223372e+18


In [37]:
# Corpus of stop words
from nltk.corpus import stopwords

In [38]:
# processes a review and returns a list of words
def review_to_words(review, string = True, remove_stopwords=True):
    # Remove HTML
    #review_text = BeautifulSoup(review).get_text()
    review_text=review
    # Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    # Convert words to lower case and split them
    words = review_text.lower().split()
    # Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    if string:
        return " ".join(words)
    else:
        return words

In [39]:
#Clean up text
#Remove non-ascii text
#Remove all rows missing reviewerName
def fixString(x):
    return x.encode('ascii',errors='ignore')

# df[["reviewText"]]=df[["reviewText"]].apply(lambda x: str(x["reviewText"]).encode('ascii',errors='ignore').decode(), axis=1)
# df[["reviewerName"]]=df[["reviewerName"]].apply(lambda x: str(x["reviewerName"]).encode('ascii',errors='ignore').decode(), axis=1)
df[["reviewText"]]=df[["reviewText"]].apply(lambda x: review_to_words(x["reviewText"]), axis=1)
df[["reviewerName"]]=df[["reviewerName"]].apply(lambda x: str(x["reviewerName"]).encode('ascii',errors='ignore').decode(), axis=1)

dfCleaned=df[df['reviewText'].str.strip().astype(bool)]
dfCleaned=dfCleaned[df['reviewerName'].str.strip().astype(bool)]

#
dfCleaned

,appID,reviewerName,reviewText,reviewerRating,reviewDate,textAnalytics
0,2297,A Google User,put imput box requires location addres called ...,0.0,27-January-2011,-9.223372e+18
1,2123,A Google User,recommend full review,0.0,10-August-2010,-9.223372e+18
2,2078,A Google User,great game played hours beat multiple times fu...,0.0,20-January-2011,-9.223372e+18
3,2078,A Google User,love game level killing suggestions full review,0.0,3-January-2011,-9.223372e+18
4,194,A Google User,constantly force close please fix used tobhave...,0.0,14-February-2011,-9.223372e+18
...,...,...,...,...,...,...
11995,109,A Google User,love view top taskbar without unlocking phone ...,1.0,17-July-2011,-9.223372e+18
11996,1758,A Google User,awesome needed full review,1.0,2012/07/10 00:00:00,-9.223372e+18
11997,1888,B. Miller,awesome app works great full review,1.0,2015/11/04 00:00:00,-9.223372e+18
11998,1061,Tumma Goutham,really awesome full review,1.0,2017/08/26 00:00:00,-9.223372e+18


In [40]:
dfCleaned.count()

appID             11947
reviewerName      11947
reviewText        11947
reviewerRating    11947
reviewDate        11947
textAnalytics     11947
dtype: int64

In [41]:
dfCleaned[dfCleaned['reviewerRating'].apply(lambda x: x==1)].count()/dfCleaned.count()

#type(df)


appID             0.165983
reviewerName      0.165983
reviewText        0.165983
reviewerRating    0.165983
reviewDate        0.165983
textAnalytics     0.165983
dtype: float64

In [42]:
dfCleaned[['reviewText']]

,reviewText
0,put imput box requires location addres called ...
1,recommend full review
2,great game played hours beat multiple times fu...
3,love game level killing suggestions full review
4,constantly force close please fix used tobhave...
...,...
11995,love view top taskbar without unlocking phone ...
11996,awesome needed full review
11997,awesome app works great full review
11998,really awesome full review


## Vectorize words
### This is based on 
https://towardsdatascience.com/sentiment-analysis-a-how-to-guide-with-movie-reviews-9ae335e6bcb2
With actual logistic regression:
https://towardsdatascience.com/sentiment-classification-with-logistic-regression-analyzing-yelp-reviews-3981678c3b44


In [43]:
# from gensim.sklearn_api import W2VTransformer

# model = W2VTransformer(size=10, min_count=1, seed=1)

In [44]:
# import statements
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import scipy
from sklearn.model_selection import train_test_split

dfReviews = dfCleaned['reviewText']
dfTrain, dfTest = train_test_split(dfReviews, test_size=.1)

# Initialize a bag of words
#vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 5000) 
vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 1000) 

# Fit transform the data 
train_feat = vectorizer.fit_transform(dfTrain).toarray()
test_feat = vectorizer.transform(dfTest).toarray()


# TFIDF train set
tfidf_transformer = TfidfTransformer().fit(train_feat)
train_tfidf = tfidf_transformer.transform(train_feat)
 
# apply tfidf to test set
test_tfidf = tfidf_transformer.transform(test_feat)

train_tfidf.todense()


matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [45]:
#rv1_tf.todense()
a=train_tfidf.todense()
print(type(a))
pddf = pd.DataFrame(data=train_tfidf.todense())
print(type(pddf))
print(a)
print(pddf)


<class 'numpy.matrix'>
<class 'pandas.core.frame.DataFrame'>
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
       0         1    2    3    4    5    6    7    8    9    ...  990  991  \
0      0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
1      0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
2      0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
3      0.0  0.263104  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
4      0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
...    ...       ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
10747  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
10748  0.0  0.284450  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
10749  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  

In [46]:
# print(a[99430,998])
# print(pddf[998][99430])

# pd.merge(pddf, dfCleaned[['reviewerRating']], left_index=True, right_index=True)

In [47]:
dfTest
rv1 = pd.Series("bad terrible")
rv1_tf = tfidf_transformer.transform(vectorizer.transform(rv1))
# individual_x=cx_t.todense()
# clf.predict_proba(individual_x)

In [48]:
featureNamesList=vectorizer.get_feature_names()
rv1_tf
cx_t = scipy.sparse.coo_matrix(rv1_tf)


print(rv1)
for i,j,v in zip(cx_t.row, cx_t.col, cx_t.data):
    if i!=0:
        break
    print("(%d, %d), %s = %s" % (i,j,featureNamesList[j], v))

0    bad terrible
dtype: object
(0, 861), terrible = 0.7893230767485331
(0, 56), bad = 0.6139780782016808


In [49]:
# individual_x=cx_t.todense()
# clf.predict(individual_x)


In [50]:
pddf

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.263104,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10747,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10748,0.0,0.284450,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10749,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10750,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
#train data
trainYdata = pd.merge(dfTrain.to_frame(), dfCleaned[['reviewerRating']], left_index=True, right_index=True)
#determined feature names
featureNamesList=vectorizer.get_feature_names()

trainYdata
dfCleaned[['reviewerRating']],dfTrain.to_frame(), trainYdata

(       reviewerRating
 0                 0.0
 1                 0.0
 2                 0.0
 3                 0.0
 4                 0.0
 ...               ...
 11995             1.0
 11996             1.0
 11997             1.0
 11998             1.0
 11999             1.0
 
 [11947 rows x 1 columns],
                                               reviewText
 7210                            sat nice app full review
 5163             going edit post app crashes full review
 847                                    great full review
 6866   giving hard time sending videos use able keeps...
 11860  great app app easy use favorite geocaching ful...
 ...                                                  ...
 5967   snooze longer works snooze used work perfectly...
 6406   able set pic wallpaper says app exists edit ev...
 1792   unintuitive controls slows phone uninstall g f...
 11792  long time user best geocaching app far full re...
 11910          great needs bit update though full review

In [52]:
# look at data for training
type(dfTrain)
stopcounter = 0
for index, value in dfTrain.items():
    print(f"Index : {index}, Value : {value}")
    stopcounter=stopcounter+1
    if stopcounter > 10:
        break

Index : 7210, Value : sat nice app full review
Index : 5163, Value : going edit post app crashes full review
Index : 847, Value : great full review
Index : 6866, Value : giving hard time sending videos use able keeps saying error would give stars fix problem full review
Index : 11860, Value : great app app easy use favorite geocaching full review
Index : 5198, Value : library access library would great worked full review
Index : 4228, Value : broken updates last versions stats type default selection unplugged shows drop menu nothing else time current available cleared cache data also uninstalled reinstalled none helped reverted last functional version full review
Index : 877, Value : unintuitive controls slows phone uninstall g full review
Index : 7089, Value : lot catching slack app hurry guys full review
Index : 1105, Value : took months completed would like levels full review
Index : 9111, Value : pretty much supposed real options buy two options needed one able turn time periods ch

In [53]:
type(train_feat)
#len(train_feat)

numpy.ndarray

In [54]:
# Dump some vectorized words and probablities
import scipy.sparse

featureNamesList=vectorizer.get_feature_names()
type(dfTrain)

cx = scipy.sparse.coo_matrix(train_feat)
cx2 = scipy.sparse.coo_matrix(train_tfidf)


print(dfTrain.iloc[0])
for i,j,v in zip(cx.row, cx.col, cx.data):
    if i!=0:
        break
    print("(%d, %d), %s = %s" % (i,j,featureNamesList[j], v))
    
print("\r\n\r\nThe weighted results\r\n")
for i,j,v in zip(cx2.row, cx2.col, cx2.data):
    if i!=0:
        break
    print("(%d, %d), %s = %s" % (i,j,featureNamesList[j], v))

sat nice app full review
(0, 36), app = 1
(0, 317), full = 1
(0, 568), nice = 1
(0, 720), review = 1


The weighted results

(0, 720), review = 0.2051345968220064
(0, 568), nice = 0.8321086750018843
(0, 317), full = 0.2051345968220064
(0, 36), app = 0.4726888483558232


In [55]:
# look at some more data
import scipy.sparse

featureNamesList=vectorizer.get_feature_names()
type(test_tfidf)

cx = scipy.sparse.coo_matrix(test_tfidf)

print(dfTest.iloc[0])
for i,j,v in zip(cx.row, cx.col, cx.data):
    if i!=0:
        break
    print("(%d, %d), %s = %s" % (i,j,featureNamesList[j], v))


great clarity wifi full review
(0, 969), wifi = 0.8483246611810449
(0, 720), review = 0.15665025330662813
(0, 349), great = 0.4809019292017903
(0, 317), full = 0.15665025330662813


### Try svm

In [56]:
# from sklearn import svm

In [57]:
# # Get trained Y data and test Y data
# trainYdata = pd.merge(dfTrain.to_frame(), dfCleaned[['reviewerRating']], left_index=True, right_index=True)
# testYdata = pd.merge(dfTest.to_frame(), dfCleaned[['reviewerRating']], left_index=True, right_index=True)
# # Get trained X data and test X data
# trainXdata = train_tfidf.todense()
# testXdata = test_tfidf.todense()

In [58]:
# # Train data - map the y to ints of scales of reviews 0,1,2,3,4 reviews
# y = trainYdata[['reviewerRating']]
# #y_int = trainYdata['reviewerRating'].apply(lambda x: 0 if x<.2 else (1 if x<.4 else (2 if x<.6 else (3 if x<.8 else 4))))
# y_int = trainYdata['reviewerRating'].apply(lambda x: x*10)
# #y_int = trainYdata['reviewerRating'].apply(lambda x: 0 if x <.5 else 1)
# y_int_test = testYdata['reviewerRating'].apply(lambda x: x*10)
# X = trainXdata

In [59]:
# clf = svm.SVC()
# clf.fit(X,y_int)



In [60]:
# clf.predict(X[:2, :])
# clf.score(X, y_int)


# clf.predict(testXdata)
# clf.score(testXdata, y_int_test)

### Modelling part
1. Leverage the raw vector count and the tf-idf weighted version

In [61]:
# # Just looking at some data
# train_tfidf
# dfTrain
# print(df.iloc[105146])
# dfTrain
# df
# train_tfidf.todense()
# dfTrain.to_frame()

In [62]:
dfTest.head(10)
#type(dfTest)

11888                       great clarity wifi full review
2105     downloaded preselected games emulated sd card ...
2115     work work followed instructions fails block ap...
4370                        access multiwindow full review
547      exactly isuppose aim touch screen galaxy full ...
10728    works perfectly dropbox rescued splashid able ...
7328     unable comment whenever view post unable type ...
10717    good new version even plays klisj p klisj corr...
9410     july latest update freezing constantly uninsta...
7055     works interface plus side works less without s...
Name: reviewText, dtype: object

In [63]:
# Get trained Y data and test Y data
trainYdata = pd.merge(dfTrain.to_frame(), dfCleaned[['reviewerRating']], left_index=True, right_index=True)
testYdata = pd.merge(dfTest.to_frame(), dfCleaned[['reviewerRating']], left_index=True, right_index=True)

In [64]:
trainYdata

,reviewText,reviewerRating
7210,sat nice app full review,0.6
5163,going edit post app crashes full review,0.4
847,great full review,0.0
6866,giving hard time sending videos use able keeps...,0.6
11860,great app app easy use favorite geocaching ful...,1.0
...,...,...
5967,snooze longer works snooze used work perfectly...,0.4
6406,able set pic wallpaper says app exists edit ev...,0.6
1792,unintuitive controls slows phone uninstall g f...,0.0
11792,long time user best geocaching app far full re...,1.0


In [65]:
# Get trained X data and test X data
trainXdata = train_tfidf.todense()
testXdata = test_tfidf.todense()


In [67]:
# print(dfCleaned[20520:20521])
# print(df1[20624:20625])
# print(testYdata[0:1])
# print(testXdata[0:1])

In [68]:
len(trainYdata.index)

10752

In [69]:
#from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

In [72]:
# Train data - map the y to ints of scales of reviews 0,1,2,3,4 reviews
y = trainYdata[['reviewerRating']]
y_int = trainYdata['reviewerRating'].apply(lambda x: 0 if x<.2 else (1 if x<.4 else (2 if x<.6 else (3 if x<.8 else 4))))
#y_int = trainYdata['reviewerRating'].apply(lambda x: x*10)
#y_int = trainYdata['reviewerRating'].apply(lambda x: 0 if x <.5 else 1)

X = trainXdata

clf = LogisticRegression(random_state=0, max_iter=1000, multi_class=True).fit(X, y_int)

# Check trained accuracy
clf.predict(X[:2, :])
clf.predict_proba(X[:2, :])
clf.score(X, y_int)
clf


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [46]:
clf.predict_proba(X[:2, :])

array([[2.53423095e-04, 1.08950810e-03, 7.45332417e-03, 9.20661122e-02,
        6.90839347e-01, 2.08298286e-01],
       [1.29446140e-04, 2.31550156e-03, 1.10806381e-03, 4.35405832e-03,
        6.45745041e-02, 9.27518426e-01]])

In [47]:
X[:2, :]

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

### Check test accuracy

In [48]:
#y_int_test = testYdata['reviewerRating'].apply(lambda x: 0 if x<.2 else (1 if x<.4 else (2 if x<.6 else (3 if x<.8 else 4))))
y_int_test = testYdata['reviewerRating'].apply(lambda x: x*10)
#y_int_test = testYdata['reviewerRating'].apply(lambda x: 0 if x<.5 else 1)

In [49]:
clf.predict(testXdata)
clf.predict_proba(testXdata)
clf.score(testXdata, y_int_test)

0.6753258508327299

In [50]:
testXdata

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [51]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 100, random_state = 42, max_depth = 10, n_jobs=-1)
# Train the model on training data
clt = rf.fit(X, y_int);

In [52]:
clf.predict(X[:2, :])
clf.predict_proba(X[:2, :])
clf.score(X, y_int)


clf.predict(testXdata)
clf.predict_proba(testXdata)
clf.score(testXdata, y_int_test)

0.6753258508327299